In [ ]:
import seaborn as sns
import pandas as pd
sns.set(font_scale=1.5)
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

### P1: Understanding Alpha

#### P1A: Using Ridge instead of LinearRegression

In HW3, we had you fit a model to predict the x-y relationship given below.

In [ ]:
p1_data = pd.read_csv("p1.csv")

In [ ]:
plt.plot(p1_data["x"], p1_data["y"])

In particular, we had you create a new dataframe with 3 features: x, sin(x) and sin(5x)

In [ ]:
featurized_p1_data = pd.DataFrame({
    "phi1": p1_data["x"],
    "phi2": np.sin(p1_data["x"]),
    "phi3": np.sin(5*p1_data["x"])
})

To understand how regularization works, let's now try using the `Ridge` module instead of `LinearRegression`. Recall that `Ridge` takes an `alpha` parameter that lets us control the complexity of our model.

Let's start by using `alpha = 0`, which will make our model work exactly like `LinearRegression`.

In [ ]:
p1_model_alpha0 = linear_model.Ridge(alpha = 0)
p1_model_alpha0.fit(featurized_p1_data, p1_data["y"])

In [ ]:
p1_model_alpha0.coef_

Observe above that the coefficients are exactly the same that we found in hw3.

Below:
1. Fit a model `p1_model_alpha100` that has an alpha value of 100. 
2. Print out the coefficients of the model and compare them to the coefficients for `p1_model`
3. Make a plot of the predictions made by `p1_model_alpha100` and compare them to the original data.

You should see that the coefficients are slightly smaller and the fit is not quite as good.

In [ ]:
p1_model_alpha100 = ... #your code here

In [ ]:
p1_model_alpha100.coef_

In [ ]:
plt.plot(p1_data["x"], p1_data["y"])
plt.plot(p1_data["x"], p1_model_alpha100.predict(featurized_p1_data))

Now repeat the same exercise with alpha = 1000.

In [ ]:
p1_model_alpha1000 = ... #your code here

In [ ]:
p1_model_alpha1000.coef_

In [ ]:
plt.plot(p1_data["x"], p1_data["y"])
plt.plot(p1_data["x"], p1_model_alpha1000.predict(featurized_p1_data))

You should see that the model has to spend much more of its "budget" on the linear coefficient, and isn't able to really capture much of the oscillating behavior.

#### P1B: Understanding Objective Functions

To get a better understanding for how $\alpha$ forces our parameters to be smaller, let's revisit the definition of our regularizaed model.

Recall that `Ridge` tries to minimize the sum of the mean squared error plus the squares of all the coefficients times alpha, i.e. $\text{MSE} + \alpha \sum_{i=1}^n \theta_i$.

For example, if our coefficients are `[2, 3, 0.5]`, then the MSE is effectively zero. That is, you should get a value that is something like 10 to the -29th power.

In [ ]:
mean_squared_error(p1_model_alpha0.predict(featurized_p1_data), p1_data["y"])

If our coefficients are `[2, 3, 0.5]` and `alpha = 100`, then the objective function is $0 + 100 \times (2^2 + 3^2 + 0.5^2) = 0$

In [ ]:
np.sum(100 * [2 * 2 + 3 * 3 + 0.5 * 0.5])

Or using `p1_model_alpha0` directly:

In [ ]:
np.sum(100 * p1_model_alpha0.coef_**2)

Below, compute the value of the objective function for alpha = 100 for `p1_model_alpha100`. Hint: The result should be approximately 856.

In [ ]:
#your code here

The gives us some insight into how `alpha` works. When we tell `p1_model_alpha100` to `fit` itself to the data, it has to minimize two things at once: The MSE and $\alpha \sum_{i=1}^n \theta_i$. So when faced with a choice between `[2, 3, 0.5]` and `[2, 2.1, 0.36]`, it picks `[2, 2.1, 0.36]`. Even though `[2, 2.1, 0.36]` has worse MSE, it has better $\alpha \sum_{i=1}^n \theta_i$.

Lastly, compute the value of the objective function for `p1_model_alpha1000` and `alpha = 1000`. You should get a value that is a little more than 4000.

In [ ]:
### your code here

### P2: Pipelines, Scaling, Regularization

In this problem, we'll explore how to use pipelines, scaling, and regularization.

#### P2A: Understanding and Fitting Our Data

In this homework, we'll be trying to fit Seattle house prices from 2014 and 2015. I originally got this data from Kaggle (https://www.kaggle.com/harlfoxem/housesalesprediction), but did some processing to remove some errors in the data (e.g. one house was incorrectly claimed to have 33 bedrooms).

In [ ]:
houses = pd.read_csv("seattle_housing.csv")

In [ ]:
houses.head(5)

For example, the fourth house sold for 604,000 U.S. Dollars, has 3 bathrooms, and 1,960 square feet of living space (182 square meters).

We see that there are a number of different features we could use to predict the house price.

In [ ]:
houses.columns

In this problem, we will use the number of bedrooms, bathrooms, square feet of living space, square foot of the lot size, condition, and grade of the house. 

In [ ]:
p2_features = ['bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'condition', 'grade']


We do not take into account other important information like the part of the city in which the town is located. For example, houses near the city center are more likely to fetch a higher value. As a bonus exercise, you can try to include this additional information after completing this entier assignment.

Using what you know, fit a linear regression model called `p2a_model` to the entire available dataset. Use the `LinearRegression` module, not the `Ridge` module.

In [ ]:
p2a_model = ... #your code here

Now compute the predicted price of each house:

In [ ]:
price_predictions = ... #your code here

Below, we compute the root mean squared error (RMSE), which is just the square root of the mean squared error. If you did everything correctly, the value should be \$206,445. This means that on average, we were somewhere around \$200,000 off from the correct price in our predictions.

In the following parts of this homework, we will try to do better, while also taking care to avoid overfitting.

In [ ]:
np.sqrt(mean_squared_error(price_predictions, houses["price"]))

#### P2B: Creating a Polynomial Model with Pipelines

One way that we can do better is to create new features which are polynomial combinations of our existing features. We did this in HW3.

In this problem you will do this again, but now using sklearn pipelines.

Using the lecture code as a guide, create and fit a model called `p2_poly2_model`. It should have two stages, the first of which is `PolynomialFeatures`, and the second of which is `LinearRegression`. Use a degree of 2 for your polynomial.

Hint: See `diamond_poly_model` in the notebook for lecture 4.

In [ ]:
p2_poly2_model = ... #your code here

Below, we compute price predictions and compute the RMSE for those predictions. 

In [ ]:
poly2_price_predictions = p2_poly2_model.predict(houses[p2_features])

In [ ]:
np.sqrt(mean_squared_error(poly2_price_predictions, houses["price"]))

Observe that the error is now lower. By creating new features, we now get better predictions.

Below, repeat the same exercise but with a degree of 3. 

In [ ]:
p2_poly3_model = ... #your code here

In [ ]:
poly3_price_predictions = p2_poly3_model.predict(houses[p2_features])

In [ ]:
np.sqrt(mean_squared_error(poly3_price_predictions, houses["price"]))

Above, you should find that the RMSE for a degree 3 model is more than \$260,000. That is, by adding additional features to use in fitting our model, we have somehow made our model worse.

In theory, adding new features should never make our Linear Regression models worse (there are interesting mathematical reasons we have not covered in our course). However, our RMSE has gotten much worse!

The reason is that the degree 3 polynomial results in values that are so large that your computer cannot store them precisely (see https://en.wikipedia.org/wiki/Round-off_error if you're curious).

For example, the line below shows that one of our features is on the order of 10 to the 18th power. 

In [ ]:
np.max(p2_poly3_model.named_steps["poly"].transform(houses[p2_features]))

In the next section, we'll see how to avoid this problem.

#### P2C: Using the StandardScaler

By rescaling the units of our original data so that each has mean 0 and variance 1, we can avoid the numerical precision errors we faced earlier.

In [ ]:
from sklearn.preprocessing import StandardScaler

Using the lecture code as a guide, create and fit a model called `p2_scaled_poly2_model`. It should have three stages, the first of which is `StandardScaler`, the second of which is `PolynomialFeatures`, and the third of which is `LinearRegression`. Use a degree of 2 for your polynomial.

Hint: See `degree_4_linear_regression_model` from the lecture 4 notebook for an example.

In [ ]:
p2_scaled_poly2_model = ... #your code here

Below we compute predictions for your model, followed by the RMSE. If you did this right, the RMSE should be around \$190,000.

In [ ]:
scaled_poly2_price_predictions = p2_scaled_poly2_model.predict(houses[p2_features])

In [ ]:
np.sqrt(mean_squared_error(scaled_poly2_price_predictions, houses["price"]))

Note that your `p2_poly2_model` from problem p2b also had an RMSE of approximately \$190,000. In other words, scaling didn't help for the degree 2 model.

However, we'll see that it helps a lot with a degree 3 polynomial. Below, create a model `p2_scaled_poly3_model` that is exactly the same as `p2_scaled_poly2_model`, but with degree 3.

In [ ]:
p2_scaled_poly3_model = ... #your code here

Below, compute the RMSE for `p2_scaled_poly3_model`. You should get a value of approximately \$185,000.

This is much better than `p2_poly3_model`, which had had RMSE of approximately \$260,000.

Here, scaling made an enormous difference.

In [ ]:
scaled_poly3_price_predictions = p2_scaled_poly3_model.predict(houses[p2_features])

In [ ]:
np.sqrt(mean_squared_error(scaled_poly3_price_predictions, houses["price"]))

#### P2D: Visualizing RMSE vs. Degree

Similar to what we did part 2D of homework 3, we will create a plot of the RMSE vs. polynomial degree.

First, we will split our data into a training, validation, and test set.

We use `np.split` to create `house_training_data`, `house_validation_data`, and `house_test_data` with 16209, 2702, and 2702 data points, respectively.

I picked these numbers so that the training set was 75% of the data, and the test and validation sets made up the remaining 25%.

In [ ]:
len(houses)

In [ ]:
N_train = 16092
N_validation = 16209 + 2682

In [ ]:
house_training_data, house_validation_data, house_test_data = ... #your code here

Fill in the function `get_training_and_validation_rmse(degree)` so that it returns the RMSE for the training and validation sets for a model with the given polynomial degree.

For example, `get_training_and_validation_rmse(4)` should return approximately `(181,000, 195,000)`.

Your solution should look quite similar to problem 2d from the lecture 3 homework.

In [ ]:
def get_training_and_validation_rmse(degree):
    ...
    #your code here, it will be long

In [ ]:
get_training_and_validation_rmse(1)

In [ ]:
get_training_and_validation_rmse(2)

In [ ]:
get_training_and_validation_rmse(3)

In [ ]:
get_training_and_validation_rmse(4)

Below, create a plot of the training and validation RMSE vs. the degree. For reference, feel free to look back at the code in the lecture 3 homework.

In [ ]:
rmses = ... #your code here

Above, you should see that the training error goes down and down as the polynomial degree increases. However, the validation error starts going up at degree 3, and increases dramatically at degree 5.

This means that we are severely overfitting once we hit degree 5.

In the next problem, we'll see how we can use regularization to keep all the degree 5 features, but still avoid overfitting.

#### P2E: Using the Validation Set to Select an Alpha

Below, create a function `get_regularized_training_and_validation_rmse(alpha)` that is exactly the same as `get_training_and_validation_rmse(degree)`, except that:

1. It should take a parameter called `alpha` instead of `degree`.
2. The pipeline should use a `PolynomialFeatures` with degree 5.
3. It should use `linear_model.Ridge` instead of `linear_model.LinearRegression`. The alpha parameter for this Ridge model should be equal to the given parameter.

In [ ]:
def get_regularized_training_and_validation_rmse(alpha):
    ...
    #your code here, it will be long

The code below will plot the training and validation RMSE for various alphas using your function. It will take a while to run, possibly a few minutes.

If it's taking too long, change the 50 to a small number.

In [ ]:
alphas = 10**np.linspace(0, 8, 50)

In [ ]:
rmses = np.array([get_regularized_training_and_validation_rmse(alpha) for alpha in alphas])

In [ ]:
plt.semilogx(alphas, rmses[:, 0])
plt.semilogx(alphas, rmses[:, 1])
plt.legend(["training", "validation"])
plt.xlabel('alpha')
plt.ylabel('RMSE')

In [ ]:
#You can use the code below to find the index of the minimum validation error
#np.where(rmses[:, 1] == min(rmses[:, 1]))

In [ ]:
alphas[21]

Some questions to ponder:

1. What is the best alpha to choose?
2. Are the models with large alpha (right side of the graph) high complexity or low complexity?
3. What part of the plot shows overfitting?

#### P2F:  Performance on the Test Set

Based on your plot from problem P2E, train a model called `p2f_model` with the optimal alpha on `house_training_data`, then compute the RMSE on the test set in `house_test_data`.

Your result should be less than \$210,000.

In [ ]:
p2f_model = ... #your code here

In [ ]:
np.sqrt(mean_squared_error(p2f_model.predict(house_test_data[p2_features]), house_test_data["price"]))

**Very important:** We managed to achieve this level of error without ever using the test data in any way. You'll see it's roughly as good as we got on the training set. Not bad! We have confidence that our model should generalize to other data from the same distribution. That is, if we pick a Seattle house at random from 2015, we would expect to get RMSE of less than \$210,000. 

In theory, we could try adjusting the alpha to get better test error. In practice, this would be an incredibly bad idea. In effect, we would be fitting our alpha to the test data. Since the goal of our model is to build something that will work for future prediction, we would have no confidence it would work in the real world.

*Note:* An RMSE of \$210,000 is much better than random guessing, but there is still a lot more work to do to get the error lower. Taking into things like the neighborhood a house belongs to would help a lot. We leave this as an exercise for the especially interested student.

#### P2G: Using RidgeCV

In problem P2F, we found the optimal alpha by using a plot of the error on a validation set.

An alternate approach is to use `RidgeCV`, which will automatically find the optimal `alpha` using only the validation set.

Create a model `p2g_model` that uses `RidgeCV`. For the alphas parameter, use `10**np.linspace(0, 8, 50)`.

This will probably take a while to run, possibly several minutes.

In [ ]:
p2g_model = ... #your code here

In [ ]:
p2g_model.named_steps['model'].alpha_

Above, we see that RidgeCV picks an alpha that is not the same as the one that you picked using a validation set.

Below, we can evaluate the performance of the resulting model. You should see that the RMSE is pretty close to what you got in problem p2f.

In [ ]:
np.sqrt(mean_squared_error(p2g_model.predict(house_test_data[p2_features]), house_test_data["price"]))

**Important note:** `RidgeCV` was able to select a good alpha without using a special validation set. That is, it picked the alpha using ONLY the traiining set.

The technique that `RidgeCV` uses is called "cross validation", and is described in lecture 4. We will not discuss the details of cross validation further in this homework.

**Caveat:** There is a subtle issue with the way we used RidgeCV. In particular, the StandardScaler scales all of the data before passing it to RidgeCV. Ideally, we'd separately scale the data for each cross validation fold. This is a pretty advanced topic, so we will not discuss this in our course.

**Takeaway:** In real world projects, you're welcome to use the approach from `p2f_model` or from `p2g_model`.

p2f_model: Set aside a special validation set and test set. Use training set to fit the parameters of your ridge model. Use the validation set to fit the hyperparameters (in this case, alpha). Use the test set to evaluate performance at the very end. 

p2g_model: Set aside a special test set. Use the cross-validation technique to fit the parameters and hyperparamters using the training data. Use the test set to evalute performance at the very end.

Both approaches are valid, but the p2g_model style is more common in the real world.

**One last warning:** It is very important to avoid using the test data in any way whatsoever! You should not use the test data to fit paramters or hyperparamters.

#### Bonus Problem P2H: Lasso

Recall from lecture that there is an alternate to Ridge regression called LASSO regression. The difference in outcome is that LASSO models have many zero paramters.

Let's see it in action. Note that LASSO models are generally more numerically difficult to fit, so you may see several warnings appear about convergence. 

In [ ]:
p2h_model = Pipeline([
    ('scale', StandardScaler()),
    ('poly', PolynomialFeatures(degree=5)),
    ('model', linear_model.LassoCV(alphas=10**np.linspace(0, 8, 50),  fit_intercept=False))
])    
p2h_model.fit(house_training_data[p2_features], house_training_data[["price"]])

You'll see the RMSE of the resulting LASSO model is similar to the RidgeCV RMSE from problem p2g.

In [ ]:
np.sqrt(mean_squared_error(p2h_model.predict(house_test_data[p2_features]), house_test_data["price"]))

The big difference is that the coefficents of the p2h_model are almost all zero, whereas with p2g_model, none of them are.

In [ ]:
p2g_model.named_steps['model'].coef_

In [ ]:
p2h_model.named_steps['model'].coef_

To understand what features each coefficient corresponds to, we can use the `get_feature_names` function of the `PolynomialFeatures` object.

In [ ]:
pf = PolynomialFeatures(degree=5)
pf.fit(house_training_data[p2_features])
feature_names = pf.get_feature_names(p2_features)
feature_names

For example, the first coefficient is approximately \$453,000, which is the intercept term for our model.

The second and third coefficients are 0, which correspond to the weight of our "bedrooms" feature and "bathrooms" feature. 

Challenge: Create a table of all the features and their weights. Only include features with non-zero weight.

In [ ]:
#your code here ...